In [ ]:
from google.colab import drive #conecta ao drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#pip install pypdf2
!pip install 'PyPDF2<3.0'
!pip install tabula-py
!pip3 install nltk
!pip install tabula-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import PyPDF2 #biblioteca que lê pdf
import re #regex
import tabula #biblioteca que lê tabelas/importa de dentro de um PDF.

In [ ]:
#Descobrir gênero através do nome
import random
import nltk
from nltk.corpus import names
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

In [ ]:
import math
import pandas as pd
import os #listar os arquivos de um diretório/pasta

In [ ]:
def char_sexo(nome):
    return {"primeira_letra" : nome[0],
            "ultima_letra": nome[-1]}

nomes =  ([(nome_m, 'M') for nome_m in names.words('male.txt')] +
          [(nome_f, 'F')  for nome_f in names.words('female.txt')])

random.shuffle(nomes) #Sorteia os dados no dataset nome.

In [ ]:
#Define status do aluno (ativo ou cancelado)
def define_status_aluno(status_pdf):
  status_aluno = status_pdf.split("Status:")
  status_aluno = status_aluno[1].split("Prorrogações:")
  status_aluno = status_aluno[0].strip()
  return status_aluno

In [ ]:
def define_tipo_escola(texto_pdf):
  dados_ensinomedio = texto_pdf.split("DADOS DO ENSINO MÉDIO")
  dados_ensinomedio = dados_ensinomedio[1].split("Página 1")
  dados_ensinomedio = dados_ensinomedio[0][0:-2]
  nome_escola = dados_ensinomedio.title()

  if "Estadual" in nome_escola:
      return "Pública"
  elif "Municipal" in nome_escola:
      return "Pública"
  elif "Federal" in nome_escola:
      return "Pública"
  elif "Pública" in nome_escola:
      return "Pública"
  elif "CEFET/MG - CAMPUS-LEOPOLDINA" in dados_ensinomedio.strip():
      return "Pública"
  elif "CEFET-MG Campus Leopoldina" in dados_ensinomedio.strip():
      return "Pública"
  elif "CEFET/MG-CAMPUS LEOPOLDINA" in dados_ensinomedio.strip():
      return "Pública"
  elif "IF SUDESTE MG - CAMPUS MURIAÉ" in dados_ensinomedio.strip():
      return "Pública"
  elif "CENTRO FEDERAL DE EDU. TECNOLÓGICA DE MINAS GERAIS" in dados_ensinomedio.strip():
      return "Pública"
  elif "IF Sudeste MG" in dados_ensinomedio.strip():
      return "Pública"
  elif "IFE-SUDESTE-CAMPUS-JUIZ DE FORA" in dados_ensinomedio.strip():
      return "Pública"
  else:
      return "Particular"

In [ ]:
def calcula_periodos_antes_saida(ano_inicial, ano_saida):
  #Calcula quantos semestres o aluno trancou.

  ano_1 = ano_inicial
  ano_2 = ano_saida
  quantidade_semestres = 0

  ano_1_ano = int(ano_1.split(".")[0]) #Pega o ano da variável ano_inicial
  ano_1_semestre = int(ano_1.split(".")[1]) #Pega o semestre da variável ano_inicial

  while(True):
    if ano_1_ano == int(ano_2.split(".")[0]) and ano_1_semestre == int(ano_2.split(".")[1]): #Se o ano e semestre do ano 1 (ano_inicial) for igual ao ano 2 (ano_saida)
      break
    if ano_1_semestre == 1:
      ano_1_semestre = ano_1_semestre + 1
    else:
      ano_1_semestre=1
      ano_1_ano = ano_1_ano + 1
    quantidade_semestres = quantidade_semestres + 1

  quantidade_semestres = quantidade_semestres
  return quantidade_semestres

In [ ]:
def extract_text_pdf(pdf):
  num_paginas = pdf.getNumPages() # Obtem quantidade de páginas do PDF
  texto = ''
  for i in range(num_paginas): # Percorre todas as páginas e obtem o texto das mesmas.
      pagina = pdf.getPage(i)
      texto += pagina.extractText()
  return texto

In [ ]:
def extract_total_horas_integralizadas(texto):
  texto_antes = texto.split("Integralizada/Pendente")[0]
  texto_depois = texto_antes.split("Exigido")[1][-74:-70]
  if texto_depois.strip().isnumeric() or not "h" in texto_depois.strip():
    return texto_depois.strip()
  else:
    return texto_depois

In [ ]:
def extract_ano_inicial_final_trancamento_tipo_saida(text_pdf):
  ano_inicial = text_pdf.split("BACHARELADO - PRESENCIAL") #linha que vem antes da informação desejada
  ano_inicial = ano_inicial[1][:7] #posição em que o dado está

  ano_saida = text_pdf.split("períodos letivos")
  ano_saida = ano_saida[0] #retirando tudo que estava antes de períodos letivos
  ano_saida = ano_saida[-8:-2] #-2 (\n) e o 0, -8 é soma de todos os caracteres

  ano_trancamento = text_pdf.split("Trancamentos")
  ano_trancamento = ano_trancamento[0][-7:] #contando espaço mais 6 caracteres
  if ano_trancamento.strip() == 'Nenhum':
     ano_trancamento = 0


  tipo_saida = text_pdf.split("Data da Colação de Grau: -")
  tipo_saida = tipo_saida[1].split("\n") #separando em cada quebra de linha (split)
  tipo_saida = tipo_saida[0] #informação desejada

  return ano_inicial[1:], ano_trancamento, tipo_saida, ano_saida

In [ ]:
def extract_qtd_reprovacao_aprovacao_media_frequencia(list_tabelas):
  nao_nulos = list_tabelas[list_tabelas['Situação'].notna()]  # Obtenho os valores da tabela não nulos de acordo com a coluna situação
  #nao_nulos = nao_nulos.drop([nao_nulos.columns[1], nao_nulos.columns[2]], axis=1)

  situacao = nao_nulos['Situação'].value_counts() # Conto as situações pela coluna "Situação" (APR e REP)
  qtd_reprovacao = 0
  qtd_reprovacao_nota = 0
  total_reprovacao_por_nota = 0
  qtd_reprovacao_falta = 0
  qtd_reprovacao_mf = 0
  qtd_reprovacao_nf = 0
  total_reprovacao_nota_e_falta = 0
  qtd_aprovacao = 0
  qtd_trancamentos = 0
  qtd_cumpridas = 0
  qtd_dispensadas = 0
  qtd_incorporado = 0
  qtd_transferido = 0

  #total de reprovações por nota
  if 'REP' in situacao:
    qtd_reprovacao = situacao['REP']
  if 'REPN' in situacao:
    qtd_reprovacao_nota = situacao['REPN']
  total_reprovacao_por_nota = qtd_reprovacao + qtd_reprovacao_nota

  #total de reprovações por falta
  if 'REPF' in situacao:
    qtd_reprovacao_falta = situacao['REPF']

  #total de reprovações por nota e falta
  if 'REPMF' in situacao:
    qtd_reprovacao_mf = situacao['REPMF']
  if 'REPNF' in situacao:
    qtd_reprovacao_nf = situacao['REPNF']
  total_reprovacao_nota_e_falta = qtd_reprovacao_mf + qtd_reprovacao_nf

  #total de aprovações
  if 'APR' in situacao:
    qtd_aprovacao = situacao['APR']
  if 'CUMP' in situacao:
    qtd_cumpridas = situacao['CUMP']
  if 'DISP' in situacao:
    qtd_dispensadas = situacao['DISP']
  if 'TRANS' in situacao:
    qtd_transferido = situacao['TRANS']
  if 'INCORP' in  situacao:
    qtd_incorporado = situacao['INCORP']
  qtd_total_aprovacoes = qtd_aprovacao + qtd_cumpridas + qtd_dispensadas + qtd_transferido + qtd_incorporado

  #total de disciplinas trancadas
  if 'TRANC' in situacao:
    qtd_trancamentos = situacao['TRANC']

  nao_nulos = nao_nulos.replace("--", 0) # Mudo todos valores que vem -- para 0

  rows_to_drop = nao_nulos[nao_nulos['Média'] == 'Média'].index
  nao_nulos = nao_nulos.drop(rows_to_drop)

  nao_nulos["Média"] = pd.to_numeric(nao_nulos["Média"], downcast="float") # Transformo os valores para float
  media = nao_nulos['Média'].mean() # Calculo a média

  nao_nulos["Freq %"] = nao_nulos["Freq %"].astype(str).str.replace(',', '.') #transformo todas as virgulas para ponto
  nao_nulos["Freq %"] = pd.to_numeric(nao_nulos["Freq %"], downcast="float")
  media_frequencia = nao_nulos['Freq %'].mean()

  return total_reprovacao_por_nota, qtd_reprovacao_falta, total_reprovacao_nota_e_falta, qtd_total_aprovacoes, qtd_trancamentos, round(float(media), 2), round(float(media_frequencia), 2)

In [ ]:
def return_columns(tabela):
  #print(len(tabela.columns))
  #display(pd.DataFrame(tabela).head())
  columns = [] # Defino uma váriavel coluna que recebe as colunas da tabela de acordo com o tipo de matéria que o aluno teve
  if len(tabela.columns) == 9:
    columns = ["Ano", "Componente Curricular", "Hora Aula", "CH", "Turma", "Freq %", "Média", "Conceito", "Situação"]
  elif len(tabela.columns) == 10:
    columns = ["Ano", "NaN1", "Componente Curricular", "Hora Aula", "CH", "Turma", "Freq %", "Média", "Conceito", "Situação"]
  elif len(tabela.columns) == 11:
    columns = ["Ano", "NaN1", "NaN2", "Componente Curricular", "Hora Aula", "CH", "Turma", "Freq %", "Média", "Conceito", "Situação"]
  else:
    columns = ["Ano", "NaN1", "NaN2", "NaN3", "Componente Curricular", "Hora Aula", "CH", "Turma", "Freq %", "Média", "Conceito", "Situação"]

  return columns

In [ ]:
def extract_table_classes(pdf, list_tabelas, arquivo):
  num_paginas = pdf.getNumPages() # Obtem quantidade de páginas do PDF
  tabela = list_tabelas[0]  #pega a primeira tabela
  # tabela.columns = tabela.iloc[0]  #recebe a primeira linha (que foi renomeada) como título
  # tabela = tabela.iloc[2:]

  columns = return_columns(tabela)

  tabela = tabela.set_axis(columns, axis=1, inplace=False) # Defino as colunas obtidas na tabela
  #table = pd.DataFrame(table[0])

  for i in range(2, num_paginas+1): # Percorre todas as páginas e obtem o texto das mesmas.
    pagina = pdf.getPage(i-1)
    texto = pagina.extractText()
    if "Ano/Período" in texto:
      table = tabula.read_pdf(arquivo, encoding='utf-8', stream=False, pages=i)
      table = table[0]
      columns = return_columns(table)
      #print(columns)
      #display(pd.DataFrame(table).columns)
      table = table.set_axis(columns, axis=1, inplace=False)
      tabela = pd.concat([tabela, table])
    #display(tabela)
  return tabela

In [ ]:
# formata o dataset para aplicar o Naive Bayes
dataset_formatado = [(char_sexo(n), s) for n,s in nomes]

# Divide os dados em train_set e test_set
train_set , test_set  = dataset_formatado[1000:], dataset_formatado[:1000]

# Cria um classificador utilizando Naive Bayes e treina o modelo utilizando os
# dados de treinamento (train_set)
classificador = nltk.NaiveBayesClassifier.train(train_set)

In [ ]:
pasta = "/content/drive/MyDrive/final/ATIVOS_E_CANCELADOS" #pega o arquivo que vai ser trabalhado
#pasta = "/content/drive/MyDrive/ColabNotebooks/Rafaela/HistóricosAtivos"

In [ ]:
def extrair_tabela_horas_aluno(pdf, arquivo):
  num_paginas = pdf.getNumPages()

  for i in range (0, num_paginas): #passa em todas as páginas do pdf
    page = read_pdf.getPage(i) #leio página por página
    texto = page.extractText() #extraio texto
    if "Carga Horária Integralizada/Pendente" in texto:
      tabelas = tabula.read_pdf(arquivo, encoding='utf-8', stream=False, pages=i+1) #caso haja o texto, leio todas as tabelas da página
      for tabela in tabelas: #percorro tabela por tabela
        value_to_search = 'Exigido'
        is_present = tabela.applymap(lambda x: x == value_to_search).any().any() # se tiver a palavra 'Exigido' na tabela obtém se os dados de acordo com a coluna
        if is_present:    # a função map mapeia os valores existentes na tabela por linha e por coluna
          tabela = tabela.dropna(axis="columns")
          #display(tabela)
          if(len(tabela.columns) > 4):
            obrigatorias = tabela.iloc[1, 1]
            optativas = tabela.iloc[1, 2]
            complementares = tabela.iloc[1, 3]
            flexibilizacao = tabela.iloc[1, 4]
          else:
            obrigatorias = 0
            optativas = 0
            complementares = 0
            flexibilizacao = 0
          return obrigatorias, optativas, complementares, flexibilizacao
          break
      break

In [ ]:
import numpy as np
import array
import re

def total_semestres_sem_disciplinas(pdf, arquivo, ano_inicial):
  num_paginas = pdf.getNumPages()
  arr = np.array([])
  # list_tabelas = tabula.read_pdf(arquivo, encoding='utf-8', stream=False, pages='1')
  # tabela = list_tabelas[0]
  # tabela = tabela['Ano/Período'].dropna().values
  # indice = np.where(tabela == 'Letivo')[0]
  # tabela = np.delete(tabela, indice)

  contador = 0

  for i in range (0, num_paginas): #passa em todas as páginas do pdf
    page = read_pdf.getPage(i) #leio página por página
    texto = page.extractText() #extraio texto
    if "Componentes Curriculares Cursados/Cursando" in texto:
      tabelas_cursados_cursando = tabula.read_pdf(arquivo, encoding='utf-8', stream=False, pages=i+1) #caso haja o texto, leio todas as tabelas da página
      tabela = tabelas_cursados_cursando[0]
      tabela = tabela.iloc[:,0].dropna().values
      indice = np.where(tabela == 'Letivo')[0]
      tabela = np.delete(tabela, indice)
      arr = np.concatenate((arr, tabela)) #juntanto os arrays

  excluir_legenda = np.where(arr == "Legenda")[0] #a função np.where irá localizar as ocorrências da string legenda no array (arr) e o 0 pega o primeiro índice da ocorrência
  if len(excluir_legenda) > 0: #se houver pelo menos uma ocorrência da string "Legenda" no array (arr), foi usado a técnica de slicing para criar um novo array que contém apenas as strings do início do array original até a primeira ocorrência da string "Legenda".
      arr = arr[:excluir_legenda[0]]

  #for element in arr:
      #print(f"{element}: {type(element)}")
  arr = arr.astype(str) #passando todos os elementos do array para string
  arr = [s.replace('#', '') for s in arr] #tirando o caracter especial
  arr = [s.replace('e', '') for s in arr]
  arr = [s.replace(' ', '') for s in arr]
  arr_sem_repeticao = list(set(arr)) #remover elementos duplicados

  semestres_curso = ["2017.1", "2017.2", "2018.1", "2018.2", "2019.1", "2019.2", "2020.1", "2020.2", "2021.1", "2021.2", "2022.1", "2022.2", "2023.1"]

  posicao = semestres_curso.index(ano_inicial) #pega a posição do ano em que o aluno entrou
  novo_array = semestres_curso[posicao:] #pega a posição

  for elemento in novo_array: #Verifica se o elemento está presente no segundo array
    if elemento not in arr_sem_repeticao: #Se o elemento estiver faltando, incrementa o contador
      contador += 1
  return contador

total_semestres_sem_disciplinas(read_pdf, os.path.join(pasta, arquivo), ano_inicial)

May 05, 2023 3:16:16 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

May 05, 2023 3:16:21 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

<ipython-input-75-e63c9b046a58>:23: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  indice = np.where(tabela == 'Letivo')[0]
May 05, 2023 3:16:24 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

<ipython-input-75-e63c9b046a58>:23: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  indice = np.where(tabela == 'Letivo')[0]


0

In [ ]:
colunas_tabela_aluno = pd.DataFrame(columns=["Total Semestres Trancado", "Semestres Sem Disciplinas", "Tipo de Escola", "Gênero", "Total de Reprovação por Nota", "Total de Reprovação por Falta", "Total de Reprovação por Nota e Falta", "Número de Discplinas Trancadas", "Número de Aprovações", "Total de Horas Obrigatórias Realizadas", "Total de Horas Optativas Realizadas", "Total de Horas Complementares Realizadas", "Total de Horas de Flexibilização Realizadas", "Status", "Ano Inicial", "Tipo Saída", "Ano/Período Letivo de Saída","Média Geral", "Média Frequência"]) #criando uma tabela vazia com cabeçalho
dados_tabela_alunos = pd.DataFrame(colunas_tabela_aluno)
display(dados_tabela_alunos)

,Total Semestres Trancado,Semestres Sem Disciplinas,Tipo de Escola,Gênero,Total de Reprovação por Nota,Total de Reprovação por Falta,Total de Reprovação por Nota e Falta,Número de Discplinas Trancadas,Número de Aprovações,Total de Horas Obrigatórias Realizadas,Total de Horas Optativas Realizadas,Total de Horas Complementares Realizadas,Total de Horas de Flexibilização Realizadas,Status,Ano Inicial,Tipo Saída,Ano/Período Letivo de Saída,Média Geral,Média Frequência


In [ ]:
for diretorio, subpastas, arquivos in os.walk(pasta):
    for arquivo in arquivos:
        print("Arquivo "+os.path.join(diretorio, arquivo))

        pdf_file = open(os.path.join(diretorio, arquivo), 'rb')
        read_pdf = PyPDF2.PdfFileReader(pdf_file)

        page = read_pdf.getPage(0)
        texto_pdf_primeira_pagina = page.extractText()

        texto_pdf = extract_text_pdf(read_pdf)

        list_tabelas = tabula.read_pdf(os.path.join(diretorio, arquivo), encoding='utf-8', stream=False, pages='1')
        list_tabelas = extract_table_classes(read_pdf, list_tabelas, os.path.join(diretorio, arquivo))

        status = define_status_aluno(texto_pdf_primeira_pagina)

        total_horas = extract_total_horas_integralizadas(texto_pdf)
        ano_inicial, ano_trancamento, tipo_saida, ano_saida = extract_ano_inicial_final_trancamento_tipo_saida(texto_pdf)

        quantidade_semestres = calcula_periodos_antes_saida(ano_inicial, "2023.1")

        nome=texto_pdf_primeira_pagina.split("Nome:")[0][12:].strip()
        nome=nome.split(" ")[0].title()
        sexo = classificador.classify(char_sexo(nome))

        tipo_escola = define_tipo_escola(texto_pdf_primeira_pagina)

        total_reprovacao_por_nota, qtd_reprovacao_falta, total_reprovacao_nota_e_falta, qtd_total_aprovacoes, qtd_trancamentos, media, media_frequencia = extract_qtd_reprovacao_aprovacao_media_frequencia(list_tabelas)

        total_obrigatorio_realizado, total_optativa_realizada, taltal_complementares, total_flexibilizacao = extrair_tabela_horas_aluno(read_pdf, os.path.join(pasta, arquivo))

        total_semestre_trancado = 0
        total_sem_disciplina = total_semestres_sem_disciplinas(read_pdf, os.path.join(diretorio, arquivo), ano_inicial)

        temp = pd.DataFrame([[ano_trancamento, total_sem_disciplina, tipo_escola, sexo, total_reprovacao_por_nota, qtd_reprovacao_falta, total_reprovacao_nota_e_falta, qtd_trancamentos, qtd_total_aprovacoes, total_obrigatorio_realizado, total_optativa_realizada, taltal_complementares, total_flexibilizacao, status, ano_inicial, tipo_saida, ano_saida, media, media_frequencia]], columns=dados_tabela_alunos.columns)
        dados_tabela_alunos = pd.concat([dados_tabela_alunos, temp])

Arquivo /content/drive/MyDrive/final/ATIVOS_E_CANCELADOS/historico_20223002872.pdf


May 05, 2023 3:16:26 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

<ipython-input-53-df9aa4593ba4>:9: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  tabela = tabela.set_axis(columns, axis=1, inplace=False) # Defino as colunas obtidas na tabela
May 05, 2023 3:16:30 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

<ipython-input-53-df9aa4593ba4>:21: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  table = table.set_axis(columns, axis=1, inplace=False)
May 05, 2023 3:16:35 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

May 05, 2023 3:16:39 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

May 05, 2023 3:16:43 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



Arquivo /content/drive/MyDrive/final/ATIVOS_E_CANCELADOS/historico_20223002907.pdf


May 05, 2023 3:16:50 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

<ipython-input-53-df9aa4593ba4>:9: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  tabela = tabela.set_axis(columns, axis=1, inplace=False) # Defino as colunas obtidas na tabela
May 05, 2023 3:16:54 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

<ipython-input-53-df9aa4593ba4>:21: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  table = table.set_axis(columns, axis=1, inplace=False)
May 05, 2023 3:16:58 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

May 05, 2023 3:17:02 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

May 05, 2023 3:17:08 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



Arquivo /content/drive/MyDrive/final/ATIVOS_E_CANCELADOS/historico_20183026523.pdf


...

May 05, 2023 4:13:20 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

<ipython-input-53-df9aa4593ba4>:9: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  tabela = tabela.set_axis(columns, axis=1, inplace=False) # Defino as colunas obtidas na tabela
May 05, 2023 4:13:25 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

<ipython-input-53-df9aa4593ba4>:21: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  table = table.set_axis(columns, axis=1, inplace=False)
May 05, 2023 4:13:30 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

May 05, 2023 4:13:34 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

May 05, 2023 4:13:37 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



Arquivo /content/drive/MyDrive/final/ATIVOS_E_CANCELADOS/historico_20223002273.pdf


May 05, 2023 4:13:44 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

<ipython-input-53-df9aa4593ba4>:9: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  tabela = tabela.set_axis(columns, axis=1, inplace=False) # Defino as colunas obtidas na tabela
May 05, 2023 4:13:47 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

<ipython-input-53-df9aa4593ba4>:21: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  table = table.set_axis(columns, axis=1, inplace=False)
May 05, 2023 4:13:51 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

May 05, 2023 4:13:58 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

May 05, 2023 4:14:01 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



Arquivo /content/drive/MyDrive/final/ATIVOS_E_CANCELADOS/historico_20183026194.pdf


May 05, 2023 4:14:05 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

<ipython-input-53-df9aa4593ba4>:9: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  tabela = tabela.set_axis(columns, axis=1, inplace=False) # Defino as colunas obtidas na tabela
May 05, 2023 4:14:11 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>

May 05, 2023 4:14:14 AM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



In [ ]:
dados_tabela_alunos_final = dados_tabela_alunos
dados_tabela_alunos_final

,Total Semestres Trancado,Semestres Sem Disciplinas,Tipo de Escola,Gênero,Total de Reprovação por Nota,Total de Reprovação por Falta,Total de Reprovação por Nota e Falta,Número de Discplinas Trancadas,Número de Aprovações,Total de Horas Obrigatórias Realizadas,Total de Horas Optativas Realizadas,Total de Horas Complementares Realizadas,Total de Horas de Flexibilização Realizadas,Status,Ano Inicial,Tipo Saída,Ano/Período Letivo de Saída,Média Geral,Média Frequência
0,0,0,Particular,M,4,0,0,0,12,600 h,0 h,48 h,0 h,ATIVO,2022.1,-,: --\n-,42.88,65.33
0,0,0,Pública,F,2,0,0,0,13,600 h,0 h,0 h,0 h,ATIVO,2022.1,-,: --\n-,49.33,70.76
0,0,0,Pública,M,16,0,11,4,18,780 h,30 h,0 h,0 h,ATIVO,2018.2,-,: --\n-,31.19,63.68
0,0,3,Pública,F,17,0,2,13,18,810 h,0 h,0 h,0 h,ATIVO,2018.2,-,: --\n-,33.71,63.84
0,0,9,Pública,M,0,0,0,0,0,0 h,0 h,0 h,0 h,CANCELADO,2018.2,SOLICITAÇÃO ESPONTÂNEA,2018.2,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0,1,Pública,M,4,2,4,0,2,90 h,0 h,0 h,0 h,ATIVO,2022.1,-,: --\n-,41.42,72.42
0,0,0,Pública,M,0,0,0,0,16,780 h,0 h,0 h,0 h,ATIVO,2022.1,-,: --\n-,56.12,66.38
0,0,0,Pública,M,0,0,0,0,16,780 h,0 h,0 h,0 h,ATIVO,2022.1,-,: --\n-,50.33,66.67
0,0,0,Particular,F,2,0,0,0,14,660 h,0 h,0 h,0 h,ATIVO,2022.1,-,: --\n-,49.52,65.43


In [ ]:
dados_tabela_alunos_final.to_csv("ativos_e_cancelados.csv", encoding='utf-8', index=False)

In [ ]:
!cp ativos_e_cancelados.csv "/content/drive/MyDrive/final/"